# Introduction

The notebook is intended to perform a Sales Forecast over the data of the [Tabular Playground Series - September 2022](https://www.kaggle.com/competitions/tabular-playground-series-sep-2022)

In [ ]:
# Import Standard Libraries
import pandas as pd

# Read Data

In [ ]:
# Read train and test data
train_data = pd.read_csv('./../../data/2022_09/train.csv')
test_data = pd.read_csv('./../../data/2022_09/test.csv')

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

# Exploratory Data Analysis

## Row ID

In [ ]:
# Check NaN values
print("Row ID NaN values count: {}".format(len(train_data[train_data['row_id'].isna()])))

In [ ]:
# Check duplicated values
print("Row ID duplicated values count: {}".format(len(test_data[test_data['row_id'].duplicated()])))

## Date

In [ ]:
# Extract day, month, year, day of the week and year_month information from 'date'
train_data['date_day'] = train_data['date'].dt.day
train_data['date_month'] = train_data['date'].dt.month
train_data['date_year'] = train_data['date'].dt.year
train_data['date_dayofweek'] = train_data['date'].dt.dayofweek
train_data['date_year_month'] = pd.to_datetime(train_data['date']).dt.to_period('M')